In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle
import re
import cv2
import time
import json
from keras.models import Model, Sequential, load_model
from keras.layers import *
from keras.preprocessing import image
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, plot_model, load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

In [ ]:
def readTextFile(file_path):
    with open(file_path) as f:
        return f.read()

doc = readTextFile("/kaggle/input/d/adityajn105/flickr8k/captions.txt")

lines = doc.split("\n")[ : -1]
lines = lines[1 : ]
for i in range(5):
    print(lines[i])
print(len(lines))

In [ ]:
descriptions = {}

for line in lines:
    tokens = line.split(',')

    img_name, caption = tokens[0], tokens[1]

    if descriptions.get(img_name) is None:
        descriptions[img_name] = []

    descriptions[img_name].append(caption)

In [ ]:
len(descriptions)

In [ ]:
base_dir = "/kaggle/input/d/adityajn105/flickr8k/Images/"
img_id = '1000268201_693b08cb0e.jpg'
img_path = os.path.join(base_dir, img_id)

if not os.path.isfile(img_path):
    print(f"File does not exist: {img_path}")
else:
    img = cv2.imread(img_path)

    if img is None:
        print(f"Failed to load image: {img_path}")
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

In [ ]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub('[^a-z]+', ' ', sentence)
    sentence = [word for word in sentence.split() if len(word) > 1]
    sentence = " ".join(sentence)

    return sentence

for img_id, caption_list in descriptions.items():
    for caption in range(len(caption_list)):
        caption_list[caption] = clean_text(caption_list[caption])

In [ ]:
with open('descriptions.txt', 'w') as f:
    f.write(str(descriptions))

In [ ]:
with open('descriptions.txt', 'r') as f:
    descriptions = f.read()

type(descriptions)

In [ ]:
json_acceptable_str = descriptions.replace("'" , "\"")

descriptions = json.loads(json_acceptable_str)
type(descriptions)

In [ ]:
total_words = []

for key, caption_list in descriptions.items():
    [total_words.append(word) for caption in caption_list for word in caption.split()]

print('Total num of words: ', len(total_words))

In [ ]:
unique_words = set()

for key in descriptions.keys():
    [unique_words.update(caption.split()) for caption in descriptions[key]]

print("Number of Unique words: ", len(unique_words))

In [ ]:
word_count_threshold = 5
freq_counts = {}

for word in total_words:
    freq_counts[word] = freq_counts.get(word , 0) + 1

print(len(freq_counts.keys()))

In [ ]:
sorted_freq_count = sorted(freq_counts.items() , reverse = True , key = lambda x:x[1])
print(sorted_freq_count[ : 5])

In [ ]:
sorted_freq_count = [x for x in sorted_freq_count if x[1] > word_count_threshold]

vocabulary = [x[0] for x in sorted_freq_count]
print("Final Vocab Size: ", len(vocabulary))

In [ ]:
image_ids = list(descriptions.keys())
print(len(image_ids))

In [ ]:
split = int(len(image_ids) * 0.8)
train_file_data = image_ids[ : split]
test_file_data = image_ids[split : ]

In [ ]:
print("No of Image Id's in Train Data: ", len(train_file_data))

In [ ]:
print("No of Image Id's in Test Data: ", len(test_file_data))

In [ ]:
train_descriptions = {}

for img_id in train_file_data:
    train_descriptions[img_id] = []

    for caption in descriptions[img_id]:
        new_caption = "startseq " + caption + " endseq"
        train_descriptions[img_id].append(new_caption)

print("Train Descriptions: ", len(train_descriptions))

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
length_dictionary = {}

for path, captions in train_descriptions.items():
    for caption in captions:
        length = len(word_tokenize(caption))
        if length not in length_dictionary:
            length_dictionary[length] = 0
        length_dictionary[length] += 1

print(length_dictionary)
sorted_by_count = sorted(length_dictionary.items(), key=lambda x: x[1], reverse=True)
count = 0
for key, value in sorted_by_count:
    print(f"length: {key} count: {value}")
    count += 1
    if count > 10 :
        break

sorted_by_length = sorted(length_dictionary.items(),key = lambda x : x[0],reverse = True)
count = 0
print("_")
for key, value in sorted_by_length:
    print(f"length: {key} count: {value}")
    count += 1
    if count > 15 :
        break

In [ ]:
model = VGG16()
model_new = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def preprocess_image(image_path):
    image = load_img(image_path, target_size = (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis = 0)
    image = preprocess_input(image)
    return image

def encode_image(image):
    image = preprocess_image(image)
    feature_vector = model_new.predict(image)
    feature_vector = feature_vector.reshape((-1, ))
    print(feature_vector.shape)
    return feature_vector

In [ ]:
# start = time.time()
# encoded_train_data = {}

# for ix, img_id in enumerate(train_file_data):
#     img_path = os.path.join(base_dir, img_id)
#     encoded_train_data[img_path] = encode_image(img_path)

#     if (ix + 1) % 100 == 0:
#         print(f"{ix + 1} images processed...")

# end = time.time()
# print("Time Elapsed: ", end - start)

In [ ]:
# start = time.time()
# encoded_test_data = {}

# for ix, img_id in enumerate(test_file_data):
#     img_path = os.path.join(base_dir, img_id)
#     encoded_test_data[img_path] = encode_image(img_path)

#     if (ix + 1) % 100 == 0:
#         print(f"{ix + 1} images processed...")

# end = time.time()
# print("Time Elapsed: ", end - start)

# with open("encoded_test_img_features.pkl", "wb") as file:
#     pickle.dump(encoded_test_data , file)

In [ ]:
with open("/kaggle/input/new-train-and-test-encodings/encoded_train_img_features.pkl", "rb") as file:
    encoded_train_data = pickle.load(file)

with open("/kaggle/input/new-train-and-test-encodings/encoded_test_img_features.pkl", "rb") as file:
    encoded_test_data = pickle.load(file)

In [ ]:
word_to_idx = {}
idx_to_word = {}

for i, word in enumerate(vocabulary):
    word_to_idx[word] = i+1
    idx_to_word[i+1] = word

idx_to_word[2573] = 'startseq'
word_to_idx['startseq'] = 2573
idx_to_word[2574] = 'endseq'
word_to_idx['endseq'] = 2574

vocab_size = len(idx_to_word) + 1
vocab_size

In [ ]:
file = open("/kaggle/input/glove6b300dtxt/glove.6B.300d.txt" , encoding = "utf8")

In [ ]:
embeddings_index = {}

for line in file:
    values = line.split()
    word = values[0]
    word_embedding = np.array(values[1: ], dtype = 'float')
    embeddings_index[word] = word_embedding

file.close()

In [ ]:
def get_embedding_matrix():
    embedding_dim = 300
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for idx, word in idx_to_word.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [ ]:
embeddings_matrix = get_embedding_matrix()
embeddings_matrix.shape

In [ ]:
max_len = 15
ip_img_features = Input(shape=(4096,))
fe1 = Dropout(0.4)(ip_img_features)
fe2 = Dense(256, activation='relu')(fe1)

ip_captions = Input(shape=(max_len,))
se1 = Embedding(vocab_size, 300, mask_zero=True)(ip_captions)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model_lstm = Model(inputs=[ip_img_features, ip_captions], outputs=outputs)
model_lstm.summary()

model_lstm.layers[2].set_weights([embeddings_matrix])
model_lstm.layers[2].trainable = False

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
max_len = 15
ip_img_features = Input(shape=(4096,))
fe1 = Dropout(0.4)(ip_img_features)
fe2 = Dense(256, activation='relu')(fe1)

ip_captions = Input(shape=(max_len,))
se1 = Embedding(vocab_size, 300, mask_zero=True)(ip_captions)
se2 = Dropout(0.4)(se1)
se3 = SimpleRNN(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model_rnn = Model(inputs=[ip_img_features, ip_captions], outputs=outputs)
model_rnn.summary()

model_rnn.layers[2].set_weights([embeddings_matrix])
model_rnn.layers[2].trainable = False

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_lstm.layers[2].set_weights([embeddings_matrix])
model_lstm.layers[2].trainable = False

In [ ]:
model_rnn.layers[2].set_weights([embeddings_matrix])
model_rnn.layers[2].trainable = False

In [ ]:
def data_generator(train_descriptions, encoded_train_data, word_to_idx, max_len, batch_size):
    x1, x2, y = [], [], []
    n = 0

    while True:
        for key, caption_list in train_descriptions.items():
            path2 = os.path.join(base_dir, key)
            photo = encoded_train_data[path2]

            for caption in caption_list:
                seq = [word_to_idx[word] for word in caption.split() if word in word_to_idx]
                for i in range(1, len(seq)):
                    input_seq, output_seq = seq[:i], seq[i]
                    input_seq = pad_sequences([input_seq], maxlen=max_len, padding='post')[0]
                    output_seq = to_categorical([output_seq], num_classes=vocab_size)[0]

                    x1.append(photo)
                    x2.append(input_seq)
                    y.append(output_seq)

                    n += 1
                    if n == batch_size:
                        yield ((np.array(x1, dtype=np.float32), np.array(x2, dtype=np.float32)), 
                               np.array(y, dtype=np.float32))
                        x1, x2, y = [], [], []
                        n = 0

In [ ]:
import tensorflow as tf
output_signature = (
    (tf.TensorSpec(shape=(None, 4096), dtype=tf.float32),  # Adjust shape according to photo encoding
     tf.TensorSpec(shape=(None, max_len), dtype=tf.float32)),
    tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32)
)

# Create a TensorFlow dataset from the generator
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_descriptions, encoded_train_data, word_to_idx, max_len, batch_size),
    output_signature=output_signature
)

batch_size = 32
steps_per_epoch = len(train_descriptions) // batch_size
epochs = 30
print("for lstm")
for epoch in range(epochs):
    model_lstm.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=1)

In [ ]:
print("for rnn")
for epoch in range(epochs):
    model_rnn.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=1)

In [ ]:
model_lstm.save('model_lstm_new_weights.h5')
model_rnn.save('model_rnn_new_weights.h5')

In [ ]:
model_lstm.load_weights('model_lstm_new_weights.h5')
model_rnn.load_weights('model_rnn_new_weights.h5')

def predict_captions(photo, value):
    ip_text = 'startseq'
    
    for i in range(max_len):
        seq = [word_to_idx[word] for word in ip_text.split() if word in word_to_idx]
        seq = pad_sequences([seq], maxlen=max_len, padding='post')

        if value == "lstm":
            pred_label = model_lstm.predict([photo, seq])
        elif value == "rnn":
            pred_label = model_rnn.predict([photo, seq])
        else:
            raise ValueError("Invalid model type. Use 'lstm' or 'rnn'.")

        pred_label = pred_label.argmax()
        pred_word = idx_to_word[pred_label]

        ip_text += " " + pred_word

        if pred_word == 'endseq':
            break

    final_caption = ip_text.split(' ')[1:-1]
    final_caption = ' '.join(final_caption)
    
    return final_caption



In [ ]:
img_path = "/kaggle/input/d/adityajn105/flickr8k/Images/3623331945_df0f51d7dd.jpg"

temp = encoded_test_data[img_path].reshape(1,4096)
print(temp)
import matplotlib.pyplot as plt
img = plt.imread(img_path)
plt.imshow(img)
plt.show()
print("with lstm")
print(predict_captions(temp,"lstm"))
print("with rnn")
print(predict_captions(temp,"rnn"))

In [ ]:
import random
from nltk.translate.bleu_score import corpus_bleu
from tqdm.notebook import tqdm


actual_lstm, predicted_lstm = [], []
actual_rnn, predicted_rnn = [], []


model_lstm.load_weights('model_lstm_new_weights.h5')
model_rnn.load_weights('model_rnn_new_weights.h5')


# encoded_keys = list(encoded_test_data.keys())[1000:1010]
# encoded_few = {key: encoded_test_data[key] for key in encoded_keys}
# for to Select a random subset of 20 images
encoded_keys = random.sample(list(encoded_test_data.keys()), 20)
encoded_few = {key: encoded_test_data[key] for key in encoded_keys}


for key in tqdm(encoded_few, leave=True):
    filename = os.path.basename(key)
    captions = descriptions[filename]
    
  
    y_pred_lstm = predict_captions(encoded_few[key].reshape(1, 4096), "lstm")
    actual_captions = [caption.split() for caption in captions]
    y_pred_lstm = y_pred_lstm.split()
    
    actual_lstm.append(actual_captions)
    predicted_lstm.append(y_pred_lstm)
    
  
    y_pred_rnn = predict_captions(encoded_few[key].reshape(1, 4096), "rnn")
    y_pred_rnn = y_pred_rnn.split()
    
    actual_rnn.append(actual_captions)
    predicted_rnn.append(y_pred_rnn)


# bleu_score_lstm = corpus_bleu(actual_lstm, predicted_lstm, weights=(0.25, 0.25, 0.25, 0.25))
# print(f'BLEU Score for LSTM: {bleu_score_lstm}')


bleu_score_1_lstm_subset = corpus_bleu(actual_lstm, predicted_lstm, weights=(1.0, 0, 0, 0))
bleu_score_2_lstm_subset = corpus_bleu(actual_lstm, predicted_lstm, weights=(0.5, 0.5, 0, 0))
bleu_score_3_lstm_subset = corpus_bleu(actual_lstm, predicted_lstm, weights=(0.33, 0.33, 0.33, 0))
bleu_score_4_lstm_subset = corpus_bleu(actual_lstm, predicted_lstm, weights=(0.25, 0.25, 0.25, 0.25))

print('BLEU scores for LSTM model (subset of 20 images):')
print('BLEU-1: %f' % bleu_score_1_lstm_subset)
print('BLEU-2: %f' % bleu_score_2_lstm_subset)
print('BLEU-3: %f' % bleu_score_3_lstm_subset)
print('BLEU-4: %f' % bleu_score_4_lstm_subset)



# bleu_score_rnn = corpus_bleu(actual_rnn, predicted_rnn, weights=(0.25, 0.25, 0.25, 0.25))
# print(f'BLEU Score for RNN: {bleu_score_rnn}')



bleu_score_1_rnn_subset = corpus_bleu(actual_rnn, predicted_rnn, weights=(1.0, 0, 0, 0))
bleu_score_2_rnn_subset = corpus_bleu(actual_rnn, predicted_rnn, weights=(0.5, 0.5, 0, 0))
bleu_score_3_rnn_subset = corpus_bleu(actual_rnn, predicted_rnn, weights=(0.33, 0.33, 0.33, 0))
bleu_score_4_rnn_subset = corpus_bleu(actual_rnn, predicted_rnn, weights=(0.25, 0.25, 0.25, 0.25))

print('BLEU scores for SimpleRNN model (subset of 20 images):')
print('BLEU-1: %f' % bleu_score_1_rnn_subset)
print('BLEU-2: %f' % bleu_score_2_rnn_subset)
print('BLEU-3: %f' % bleu_score_3_rnn_subset)
print('BLEU-4: %f' % bleu_score_4_rnn_subset)
